In [19]:
# Import the required packages
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

### Análisis general de distintos modelos

In [ ]:
#pip install pycaret

In [2]:
import pycaret.classification

In [3]:
# De esta forma podemos realizar custom transformer
# Todas las transformaciones deberían tener su propio transformer

class SelectColumnsTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, **transform_params):
        cpy_df = X[self.columns].copy()
        return cpy_df

    def fit(self, X, y=None, **fit_params):
        return self

In [4]:
train_df = pd.read_csv("../data/travel_insurance_prediction_train.csv")
test_df = pd.read_csv("../data/travel_insurance_prediction_test.csv")

Explorar preliminarmente todos los modelos de clasificación posibles, para hacer un primer descarte. Tomando parámetros por default.

In [5]:
from pycaret.classification import *
clf = setup(data = train_df, target = 'TravelInsurance')

,Description,Value
0,session_id,4480
1,Target,TravelInsurance
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(1490, 10)"
5,Missing Values,False
6,Numeric Features,2
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
#best = compare_models()
best = compare_models(sort = 'F1') #default is 'Accuracy', LO CAMBIO POR F1 QUE ES LA INDICADA COMO REFERENCIA

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8157,0.8031,0.5792,0.8444,0.6846,0.5621,0.5830,1.0180
gbc,Gradient Boosting Classifier,0.8110,0.8017,0.5628,0.8483,0.6740,0.5494,0.5735,0.0650
xgboost,Extreme Gradient Boosting,0.7821,0.7919,0.6012,0.7320,0.6585,0.5012,0.5076,0.2500
knn,K Neighbors Classifier,0.7946,0.7675,0.5654,0.7948,0.6549,0.5161,0.5342,0.0220
lightgbm,Light Gradient Boosting Machine,0.7831,0.7859,0.5931,0.7366,0.6546,0.5000,0.5078,0.1770
rf,Random Forest Classifier,0.7793,0.7847,0.5959,0.7245,0.6521,0.4935,0.4995,0.1500
dt,Decision Tree Classifier,0.7639,0.7400,0.6122,0.6871,0.6448,0.4693,0.4731,0.0100
ada,Ada Boost Classifier,0.8004,0.7718,0.5053,0.8712,0.6332,0.5124,0.5500,0.0670
et,Extra Trees Classifier,0.7563,0.7568,0.5877,0.6765,0.6278,0.4483,0.4515,0.1510
lda,Linear Discriminant Analysis,0.7822,0.7708,0.5164,0.7886,0.6183,0.4780,0.5009,0.0300


### Dividimos los datos en train y validation 

Para tener la posibilidad de evaluar los diferentes modelos en un conjunto de validación

In [7]:
# The data for training the models
X_train = train_df.drop(columns=["Customer", "TravelInsurance"])
y_train = train_df["TravelInsurance"].values

In [8]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [9]:
X_train_.shape, X_val.shape, y_train_.shape, y_val.shape

((1192, 8), (298, 8), (1192,), (298,))

**Repaso el modelo baseline**

Pero con la división de grupo de train reducido y validación, ahora.

In [10]:
#Pre-procesamiento transformaciones
transformer = make_column_transformer(
    (KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile"), ["Age", "AnnualIncome"]),
    (OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"),
     ["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]),
    remainder="passthrough")

#ChronicDiseases no fue icluida en el OneHotEncoder por que ya esté encodeada en 0-1.
#Usó absolutamente todas las variables del dataset

In [12]:
X_train_p=transformer.fit_transform(X_train_)   #aplico sobre todas las columnas 
X_val_p=transformer.transform(X_val)   

In [13]:
#Los parámetros del modelo baseline son
baseline_params={
    'criterion': 'entropy', 
    'max_depth': 10, 
    'min_samples_leaf': 5}

In [14]:
tree_baseline = DecisionTreeClassifier(criterion="entropy", max_depth= 10, min_samples_leaf= 5, random_state=42)
tree_baseline.fit(X_train_p, y_train_)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [15]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, tree_baseline.predict(X_train_p)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, tree_baseline.predict(X_val_p)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.84      0.95      0.89       774
           1       0.87      0.66      0.75       418

    accuracy                           0.85      1192
   macro avg       0.85      0.80      0.82      1192
weighted avg       0.85      0.85      0.84      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       184
           1       0.77      0.66      0.71       114

    accuracy                           0.80       298
   macro avg       0.79      0.77      0.78       298
weighted avg       0.79      0.80      0.79       298



### Otros modelos para analizar este problema de clasificación

In [20]:
#Pre-procesamiento con pipelines para hacer más sencillas las variantes (idem versión anterior)
variables_num=["Age", "AnnualIncome"]
variables_cat=["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]

Xtrain=X_train_[variables_num+variables_cat]
Xval=X_val[variables_num+variables_cat]

transf_num=Pipeline([('select_num_columns', SelectColumnsTransformer(variables_num)),
                    ('kbins_discretizer', KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile"))      
                    ])

transf_cat=Pipeline([('select_cat_columns', SelectColumnsTransformer(variables_cat)),
                    ('one_hot_enc', OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"))      
                    ])

pipeline_transformer = ColumnTransformer([('num', transf_num, variables_num),
                                        ('cat', transf_cat, variables_cat)
                                        ])

train = pipeline_transformer.fit_transform(Xtrain)
val = pipeline_transformer.fit_transform(Xval)

In [26]:
#Todas las alternativas de pre-procesamiento con pipelines 

##1er paso: filtras qué variables vamos a considerar en el modelo
variables_num=["Age", "AnnualIncome"]
#variables_cat=["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad","ChronicDiseases"]
variables_cat=["Employment Type", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]

##reduce el dataset en función de las columnas seleccionadas
Xtrain=X_train_[variables_num+variables_cat]
Xval=X_val[variables_num+variables_cat]

##2do paso: programa las transformaciones según tipo de variable
transf_num=Pipeline([('select_num_columns', SelectColumnsTransformer(variables_num)),
                    ('kbins_discretizer', KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")),
                   #  ("standard_scaler", StandardScaler()),
                   #  ("min_max_scaler", MinMaxScaler())
                    ])

transf_cat=Pipeline([('select_cat_columns', SelectColumnsTransformer(variables_cat)),
                    ('one_hot_enc', OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"))      
                    ])

pipeline_transformer = ColumnTransformer([('num', transf_num, variables_num),
                                        ('cat', transf_cat, variables_cat)
                                        ])

##3er paso: ejecuta la transformación de manera conjunta
train = pipeline_transformer.fit_transform(Xtrain)
val = pipeline_transformer.fit_transform(Xval)

In [27]:
tree_baseline_ = DecisionTreeClassifier(criterion="entropy", max_depth= 10, min_samples_leaf= 5, random_state=42)
tree_baseline_.fit(train, y_train_)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [28]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, tree_baseline_.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, tree_baseline_.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.82      0.95      0.89       774
           1       0.88      0.62      0.73       418

    accuracy                           0.84      1192
   macro avg       0.85      0.79      0.81      1192
weighted avg       0.84      0.84      0.83      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       184
           1       0.82      0.61      0.70       114

    accuracy                           0.80       298
   macro avg       0.81      0.77      0.78       298
weighted avg       0.80      0.80      0.79       298



Eliminando del modelo dos variables que no serían muy necesarias, los resultados se mantienen relativamente iguales

**Modelo SGD**

El modelo SGD Clasifier requiere que los variables numéricas estén estandarizadas, modifico el pipeline numérico en función de eso.

In [37]:
#Todas las alternativas de pre-procesamiento con pipelines 

##1er paso: filtras qué variables vamos a considerar en el modelo
variables_num=["Age", "AnnualIncome"]
variables_cat=["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad","ChronicDiseases"]
#variables_cat=["Employment Type", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]

##reduce el dataset en función de las columnas seleccionadas
Xtrain=X_train_[variables_num+variables_cat]
Xval=X_val[variables_num+variables_cat]

##2do paso: programa las transformaciones según tipo de variable
transf_num=Pipeline([('select_num_columns', SelectColumnsTransformer(variables_num)),
                   # ('kbins_discretizer', KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")),
                     ("standard_scaler", StandardScaler()),
                   #  ("min_max_scaler", MinMaxScaler())
                    ])

transf_cat=Pipeline([('select_cat_columns', SelectColumnsTransformer(variables_cat)),
                    ('one_hot_enc', OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"))      
                    ])

pipeline_transformer = ColumnTransformer([('num', transf_num, variables_num),
                                        ('cat', transf_cat, variables_cat)
                                        ])

##3er paso: ejecuta la transformación de manera conjunta
train = pipeline_transformer.fit_transform(Xtrain)
val = pipeline_transformer.fit_transform(Xval)

In [40]:
#Entrenamiento de la versión más sencilla del modelo, con parámetros por default
sgd = SGDClassifier(random_state=42)
sgd.fit(train, y_train_)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [41]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, sgd.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, sgd.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.76      0.92      0.83       774
           1       0.75      0.47      0.58       418

    accuracy                           0.76      1192
   macro avg       0.76      0.69      0.71      1192
weighted avg       0.76      0.76      0.74      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.73      0.92      0.81       184
           1       0.77      0.44      0.56       114

    accuracy                           0.73       298
   macro avg       0.75      0.68      0.68       298
weighted avg       0.74      0.73      0.71       298



No pareciera estar ofreciendo mejores resultados que el Árbol de Decisión. Vamos a probar optimizando los hiperparámetros, haciendo un GridSearchCV.

In [62]:
#Grilla de parámetros
params={'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        'penalty':['l1','l2','elasticnet'],
      # 'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive']
       }

sgd_= SGDClassifier(random_state=42)

In [63]:
#Búsqueda de parámetros
cv_sgd = GridSearchCV(sgd_, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_sgd.fit(train, y_train_)

GridSearchCV(cv=5, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5, random_state=42,
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'loss': ['hinge', 'log', 'modified_huber',
                                  'squared_hinge', 'perceptron'],
     

In [64]:
cv_sgd.best_estimator_

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [65]:
cv_sgd.best_params_

{'alpha': 0.001, 'loss': 'log', 'penalty': 'elasticnet'}

In [66]:
#Entrenamiento de la mejor versión encontrada del modelo
sgd = SGDClassifier(random_state=42,alpha=0.001, loss='log', penalty='elasticnet' )
sgd.fit(train, y_train_)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [67]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, sgd.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, sgd.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.76      0.94      0.84       774
           1       0.80      0.46      0.59       418

    accuracy                           0.77      1192
   macro avg       0.78      0.70      0.71      1192
weighted avg       0.77      0.77      0.75      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.74      0.95      0.83       184
           1       0.85      0.45      0.59       114

    accuracy                           0.76       298
   macro avg       0.79      0.70      0.71       298
weighted avg       0.78      0.76      0.74       298



La mejor opción de este modelo, encontrada por medio del GridSearchCV mejora las predicciones sobre el conjunto de validación en relación al modelo con parámetros por default pero continúa sin ser una mejor alternativa que el Árbol de Decisión.

**Modelo Random Forest Classifier**

Voy a hacer el primer entrenamiento de este modelo, en su versión más simple, con las variables preprocesadas del mismo modo que en el modelo tomado como baseline

In [68]:
#Todas las alternativas de pre-procesamiento con pipelines 

##1er paso: filtras qué variables vamos a considerar en el modelo
variables_num=["Age", "AnnualIncome"]
variables_cat=["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad","ChronicDiseases"]
#variables_cat=["Employment Type", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]

##reduce el dataset en función de las columnas seleccionadas
Xtrain=X_train_[variables_num+variables_cat]
Xval=X_val[variables_num+variables_cat]

##2do paso: programa las transformaciones según tipo de variable
transf_num=Pipeline([('select_num_columns', SelectColumnsTransformer(variables_num)),
                    ('kbins_discretizer', KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")),
                   #  ("standard_scaler", StandardScaler()),
                   #  ("min_max_scaler", MinMaxScaler())
                    ])

transf_cat=Pipeline([('select_cat_columns', SelectColumnsTransformer(variables_cat)),
                    ('one_hot_enc', OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"))      
                    ])

pipeline_transformer = ColumnTransformer([('num', transf_num, variables_num),
                                        ('cat', transf_cat, variables_cat)
                                        ])

##3er paso: ejecuta la transformación de manera conjunta
train = pipeline_transformer.fit_transform(Xtrain)
val = pipeline_transformer.fit_transform(Xval)

In [70]:
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(train, y_train_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [71]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, rfc.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, rfc.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       774
           1       0.92      0.77      0.84       418

    accuracy                           0.90      1192
   macro avg       0.90      0.87      0.88      1192
weighted avg       0.90      0.90      0.89      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       184
           1       0.76      0.63      0.69       114

    accuracy                           0.78       298
   macro avg       0.78      0.75      0.76       298
weighted avg       0.78      0.78      0.78       298



 Vamos a probar optimizando los hiperparámetros, haciendo un GridSearchCV.

In [72]:
#Grilla de parámetros
params={'n_estimators':[4, 6, 8 , 10 , 12],
        'criterion':['gini','entropy'],
        'max_depth':[6, 8, 10, 12],
        'min_samples_leaf':[2, 4, 5, 8],
       }

rfc_= RandomForestClassifier(random_state=42)

In [73]:
#Búsqueda de parámetros
cv_rfc = GridSearchCV(rfc_, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_rfc.fit(train, y_train_)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [74]:
cv_rfc.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [77]:
cv_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'n_estimators': 10}

In [78]:
#Entrenamiento de la mejor versión encontrada del modelo
rfc = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth= 10, min_samples_leaf= 2, random_state=42)
rfc.fit(train, y_train_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [79]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train_, rfc.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, rfc.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.83      0.97      0.89       774
           1       0.91      0.64      0.75       418

    accuracy                           0.85      1192
   macro avg       0.87      0.80      0.82      1192
weighted avg       0.86      0.85      0.84      1192

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       184
           1       0.82      0.61      0.70       114

    accuracy                           0.80       298
   macro avg       0.81      0.76      0.77       298
weighted avg       0.80      0.80      0.79       298

